In [1]:
import scipy.io
import numpy as np
import os
import scipy.signal as signal
import tqdm

from scipy.signal import butter, lfilter

In [81]:
def butter_bandpass_filter(lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs

    low = lowcut / nyquist
    high = highcut / nyquist

    b, a = butter(order, [low, high], btype='band')
    return b, a

def apply_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass_filter(lowcut, highcut, fs, order)
    return lfilter(b, a, data, axis=0)

def load_data(path, lowcut= 8, highcut = 30) -> tuple:

    data = scipy.io.loadmat(path)

    # mentioned in bci comp docs
    cnt = data['cnt'].astype(np.float32) * 0.1

    fs = data['nfo']['fs'][0,0]

    cnt = apply_filter(cnt, lowcut, highcut, fs)

    mrk_pos = data['mrk']['pos'][0][0].flatten()
    mrk_label = data['mrk']['y'][0][0].flatten()

    cue_min, cue_max = 0.5, 2.5
    cue_min_samples = int(cue_min * fs)
    cue_max_samples = int(cue_max * fs)
    window = cue_max_samples - cue_min_samples

    num_trials = len(mrk_pos)
    num_channels = cnt.shape[1]
    
    X= np.zeros((num_trials, num_channels, window))

    for i, pos in enumerate(mrk_pos):
        start, end = pos + cue_min_samples, pos + cue_max_samples

        if end > cnt.shape[0]:
            break

        X[i] = cnt[start:end, :].T

    
    y = mrk_label[:X.shape[0]]

    return (X, y, fs)


In [82]:

file_path_parent = 'data/bca_3_4a'
abs_path = os.path.abspath(file_path_parent)
#print(abs_path)

paths = []
for root, _, files in os.walk(abs_path):

    for file in files:
        paths.append(os.path.join(root,file))


In [83]:
data_per_subject = []

for path in paths:
    
    # tuple of (X, y, fs)
    data = load_data(path)
    data_per_subject.append(data) 
    
print(len(data_per_subject))


5


In [102]:
import pandas as pd
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

test_data = data_per_subject[0]
X = test_data[0]
print(X.shape)

y = test_data[1]
print(y.shape)

(280, 118, 200)
(280,)


In [ ]:
valid_indices = ~np.isnan(y)

x_train = X[valid_indices]
y_train = y[valid_indices]

#trials, channels, window
print(x_train.shape)
print(y_train.shape)

(168, 118, 200)
(168,)


In [ ]:
csp_per_class = 6

csp = CSP(n_components=6)

X_csp = csp.fit_transform(x_train, y_train)

print("Transformed CSP data shape:", X_csp.shape)


Computing rank from data with rank=None
    Using tolerance 2.4e+02 (2.2e-16 eps * 118 dim * 9.1e+15  max singular value)
    Estimated rank (data): 118
    data: rank 118 computed from 118 data channels with 0 projectors
Reducing data rank from 118 -> 118
Estimating class=1.0 covariance using EMPIRICAL
Done.
Estimating class=2.0 covariance using EMPIRICAL
Done.
Transformed CSP data shape: (168, 6)
[[-1.89976316 -1.04884503 -1.4116893  -1.14594952 -1.31279263 -1.34156192]
 [-1.90123068 -1.39526849 -1.56683859 -1.64703025 -1.08244386 -1.20927387]
 [-1.28162918 -1.29046111 -1.28847043 -1.51150397 -0.98021213 -0.59321028]
 ...
 [-0.23205413 -0.31806312 -0.49579803 -0.32400989  0.1445882   0.14219327]
 [-0.38006421 -0.68336429 -0.20181244 -0.89976281 -0.27888951 -0.32642536]
 [-0.82478281 -0.80374008 -0.0699445  -0.77148281 -0.68607223 -0.48662077]]


In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_csp, y_train, test_size=0.2, random_state=4,
)


In [117]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)  # Train LDA
y_pred = lda.predict(X_test)  # Predict

# Calculate accuracy
accuracy = np.mean(y_pred == y_test) * 100
print(f"LDA Classification Accuracy: {accuracy:.2f}%")

LDA Classification Accuracy: 88.24%


In [119]:
X_df = pd.DataFrame(X_csp)
y_df = pd.DataFrame(y_train)